In [1]:
# import numpy as np
# df=np.load("/Users/rohitviswam/Downloads/CSI-02-0005/H_32T4R_30_1RB.npy")
# data=df.reshape(2100,398,256)
# data.shape
# feature_len=data.shape[-1]

In [2]:

import scipy.io
# Load the .mat file
file_path = '../../EV_Rank_1_52_RBs_50_UEs_1000_snaps.mat'
data = scipy.io.loadmat(file_path)

# Extract the relevant data
EV_data = data['EV_re_im_split']

# Check the shape and structure of the extracted data
data = EV_data
del EV_data
data.shape

(50, 1000, 832)

In [3]:
# data = data[:5]

In [4]:
data.shape

(50, 1000, 832)

In [5]:
import numpy as np
import torch
from iTransformer import iTransformer
from iTransformer import iTransformer2D
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
# data_flattened = data.reshape(data.shape[0], -1)  # shape becomes (2100, 398*256)


In [ ]:
# data = data_flattened

In [ ]:
import numpy as np
import torch

# Assuming `data` is already loaded and is of shape (2100, 398, 256)

def preprocess_UE_data(data, lookback_len=10):
    num_UEs, num_timesteps, num_features = data.shape
    input_list = []
    target_list = []

    for ue in range(num_UEs):
        ue_data = data[ue]
        for i in range(num_timesteps - lookback_len):
            input_seq = ue_data[i: i + lookback_len]
            target_seq = ue_data[i + lookback_len]
            
            input_list.append(input_seq)
            target_list.append(target_seq)

    inputs = torch.tensor(input_list, dtype=torch.float32)
    targets = torch.tensor(target_list, dtype=torch.float32)

    return inputs, targets

# Usage example
inputs, targets = preprocess_UE_data(data, lookback_len=10)

print("Input shape:", inputs.shape)   # Expected: (samples, 10, 256)
print("Target shape:", targets.shape) # Expected: (samples, 256)


In [ ]:
import torch
import torch.nn as nn
from iTransformer import iTransformer

# Set up parameters and data
num_UEs, num_timesteps, num_features = data.shape
lookback_len = 10
forecast_len = 1


# Define the iTransformer model
model = iTransformer2D(
    num_variates=num_features,
    lookback_len=lookback_len,
    dim=832,           # Model dimension
    depth=10,           # Number of layers
    heads=16,           # Number of attention heads
    dim_head=128,       # Dimension per head
    pred_length=forecast_len,  # Prediction horizon (1 in this case)
    num_time_tokens=2,  # Single token per variate
    use_reversible_instance_norm=True
)



In [ ]:
from torchinfo import summary

# Assuming your model object is already defined as `model`

# Generate a model summary (batch size and input dimensions as required)
model_summary = summary(model, input_size=(4, 10, 832))  # Adjust batch size and input size accordingly

# This will print a detailed summary with parameters, layers, and dimensions
print(model_summary)


In [ ]:
# Print the architecture by directly printing the model object
print(model)


In [ ]:
# Define training settings
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cuda")

model.to(device)

inputs = inputs.to(device)
targets = targets.to(device)

# Loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training Loop
epochs = 10
batch_size = 32

for epoch in range(epochs):
    model.train()
    epoch_loss = 0.0
    num_batches = int(len(inputs) / batch_size)
    
    for batch in range(num_batches):
     batch_inputs = inputs[batch * batch_size:(batch + 1) * batch_size]
     batch_targets = targets[batch * batch_size:(batch + 1) * batch_size]

     optimizer.zero_grad()
     preds = model(batch_inputs)[forecast_len].squeeze(1)  # Remove the singleton dimension
     loss = criterion(preds, batch_targets)
     loss.backward()
     optimizer.step()
     epoch_loss += loss.item()


    print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss / num_batches:.4f}")


In [ ]:
import pandas as pd

def tabulate_predictions(inputs, targets, model, forecast_len=1, batch_size=64):
    model.eval()
    all_preds = []
    all_targets = []

    num_batches = int(len(inputs) / batch_size)

    with torch.no_grad():
        for batch in range(num_batches):
            batch_inputs = inputs[batch * batch_size:(batch + 1) * batch_size]
            batch_targets = targets[batch * batch_size:(batch + 1) * batch_size]

            # Predict using the model
            preds = model(batch_inputs)[forecast_len].squeeze(1)
            all_preds.extend(preds.cpu().numpy())
            all_targets.extend(batch_targets.cpu().numpy())

    # Create DataFrame for easier tabulation and analysis
    df_results = pd.DataFrame({
        'Actual': [list(target) for target in all_targets],
        'Predicted': [list(pred) for pred in all_preds]
    })

    return df_results

# Example Usage:
results_df = tabulate_predictions(inputs, targets, model, forecast_len=1, batch_size=64)
print(results_df.head())  # Print the first few rows to verify


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_predictions(inputs, targets, model, forecast_len=1, num_samples=10, batch_size=64):
    model.eval()
    predictions = []
    actuals = []

    num_batches = min(int(len(inputs) / batch_size), num_samples)

    with torch.no_grad():
        for batch in range(num_batches):
            batch_inputs = inputs[batch * batch_size:(batch + 1) * batch_size]
            batch_targets = targets[batch * batch_size:(batch + 1) * batch_size]

            preds = model(batch_inputs)[forecast_len].squeeze(1)
            predictions.extend(preds.cpu().numpy())
            actuals.extend(batch_targets.cpu().numpy())

    predictions = np.array(predictions)
    actuals = np.array(actuals)

    # Plot each sample
    plt.figure(figsize=(15, 8))
    for i in range(num_samples):
        plt.subplot(num_samples // 2, 2, i + 1)
        plt.plot(actuals[i], label='Actual')
        plt.plot(predictions[i], label='Predicted')
        plt.legend()
        plt.xlabel('Feature Index')
        plt.ylabel('Value')
        plt.title(f'Sample {i + 1}')

    plt.tight_layout()
    plt.show()

plot_predictions(inputs, targets, model, forecast_len=1, num_samples=2, batch_size=64)
